<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/3f/HubSpot_Logo.svg/220px-HubSpot_Logo.svg.png" alt="drawing" width="200" align='left'/>

# Hubspot - Create contacts from linkedin post likes
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Hubspot/Hubspot_create_contacts_from_linkedin_post_likes.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

#hubspot #crm #sales #linkedin #post #contact

## Input

### Install librairies

In [20]:
from naas_drivers import linkedin, hubspot
import naas
import requests

### Setup your secrets
Uncomment lines to setup.<br>
<a href='https://docs.naas.ai/features/secret'>How to setup your secret keys ?</a>

#### → For Hubspot

In [21]:
#naas.secret.add('HUBSPOT_API_KEY', secret="7696655f-3aff-4524-84f1-5c988abdab9f")

#### → For LinkedIn

<a href='https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75'>How to get your cookies ?</a>

In [22]:
#naas.secret.add('LINKEDIN_LI_AT', secret="***")# EXAMPLE AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2
#naas.secret.add('LINKEDIN_JSESSIONID', secret= "ajax:***") # EXAMPLE ajax:8379907400220387585

### Configure secrets to connect to APIs

In [23]:
#Hubspot API
auth_token = naas.secret.get('HUBSPOT_API_KEY')

In [24]:
# LinkedIn API
LI_AT = naas.secret.get('LINKEDIN_LI_AT') 
JSESSIONID = naas.secret.get('LINKEDIN_JSESSIONID')  

### Enter post URL

In [25]:
POST_URL = "----"

### Get post likes

In [27]:
df_posts = linkedin.connect(LI_AT, JSESSIONID).post.get_likes(POST_URL)

# Display the number of likes
print("Number of likes: ", df_posts.PROFILE_URN.count())

In [ ]:
# Show dataframe with list of profiles from likes 
df_posts

## Model

### Create contacts from LinkedIn post likes

In [46]:
def create_contacts_from_post(df,
                              c_profile_urn="PROFILE_URN",
                              c_firstname="FIRSTNAME",
                              c_lastname="LASTNAME",
                              c_occupation="OCCUPATION"):
    for _, row in df.iterrows():
        profile_urn = row[c_profile_urn]
        firstname = row[c_firstname]
        lastname = row[c_lastname]
        occupation = row[c_occupation]
        linkedinbio = f"https://www.linkedin.com/in/{profile_urn}"
        email = None
        phone = None

        # contact
        try:
            contact = linkedin.connect(LI_AT, JSESSIONID).profile.get_contact(linkedinbio)
            email = contact.loc[0, "EMAIL"]
            phone = contact.loc[0, "PHONENUMBER"]
        except:
            print("No contact info")

        # With send method
        data = {"properties": 
                {
                    "linkedinbio": linkedinbio,
                    "firstname": firstname,
                    "lastname": lastname,
                    "jobtitle": occupation,
                    "email": email,
                    "phone": phone,
                }
               }
        print(data)
        hubspot.connect(auth_token).contacts.send(data)

## Output

### Display result

In [ ]:
create_contacts_from_post(df_posts)